In [1]:
import torch
import numpy as np
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)
import matplotlib.pyplot as plt
import scipy.misc
from PIL import Image
import json
%matplotlib inline


/home/seongha/anaconda3/envs/MGSSL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


coco

In [2]:
path_csv = '../data/coco'

from collections import defaultdict, Counter
class_num = defaultdict(int)
with open(path_csv + '/data/train_anno.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    for l in each['labels']:
      class_num[l] += 1

  nums = gt_labels.sum(axis=0)
  adj = []
  for i,col in enumerate(gt_labels.T):
    if i in [34]:
      print(i)
      selected = gt_labels[np.isin(col, [1.0]), :]
      nonzero_cnt = (selected != 0).sum(1)
      cnter = Counter(nonzero_cnt)
      print(cnter)
    cond_prob = gt_labels[np.isin(col,[1.0]),:].sum(axis=0)
    cond_prob[i] = 0
    adj.append(cond_prob)
    # print(adj[-1])
  nums = nums.tolist()
  nums.sort()
  nums.reverse()
  # nums = reversed(nums)
  print(max(nums), min(nums))
  di={'adj': np.asarray(adj), "nums": np.asarray(nums)}
  class_di = {k: v for k, v in sorted(class_num.items(), key=lambda item: item[1], reverse=True)} #sorted
print(class_di.keys(), class_di.values())

82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
34
Counter({1: 1234, 2: 407, 3: 116, 4: 27, 5: 8, 8: 3, 7: 2, 6: 1})
45174.0 128.0
dict_keys([49, 22, 18, 27, 26, 13, 14, 36, 74, 2, 8, 12, 21, 57, 75, 24, 23, 41, 51, 28, 62, 72, 20, 16, 76, 68, 7, 32, 77, 58, 61, 42, 73, 44, 67, 65, 70, 9, 0, 10, 59, 50, 53, 11, 17, 37, 47, 5, 4, 34, 78, 52, 54, 64, 40, 3, 30, 33, 66, 39, 25, 15, 79, 45, 46, 63, 31, 19, 1, 60, 56, 43, 29, 38, 71, 55, 6, 48, 69, 35]) dict_values([45174, 8950, 8606, 8378, 6518, 5968, 5028, 4861, 4321, 3924, 3844, 3734, 3322, 3291, 3191, 3170, 3159, 3097, 3084, 3041, 2986, 2893, 2818, 2791, 2749, 2667, 2539, 2537, 2530, 2511, 2493, 2475, 2464, 2442, 2368, 2343, 2317, 2287, 2243, 2241, 2209, 2202, 2180, 2098, 2080, 2068, 2003, 1884, 1804, 1798, 1771, 1671, 1645, 1631, 1625, 1618, 1518, 1511, 1510, 1471, 1389, 1340, 1324, 1290, 1216, 1214, 1205, 1186, 1171, 1170, 1105, 1089, 1062, 821, 700, 673, 668, 481, 151, 128])


In [3]:
test_dataset = COCO2014('../data/coco', phase='val', inp_name='../data/coco/coco_glove_word2vec.pkl')
# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
# test_dataset = Voc2007Classification('data/voc', 'test', inp_name='data/voc/voc_glove_word2vec.pkl')
# train_dataset = COCO2014('data/coco', phase='train', inp_name='data/coco/coco_glove_word2vec.pkl')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
test_dataset.transform = transforms.Compose([
                MultiScaleCrop(224, scales=(1.0, 0.875, 0.75, 0.66, 0.5), max_distort=2),
                transforms.ToTensor(),
                normalize,
            ])

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
from util import AveragePrecisionMeter
AP = AveragePrecisionMeter(difficult_examples=False)



[dataset] Done!
[annotation] Done!
[json] Done!


In [4]:
from models import *
from backbones.config import config
import pathlib

print(config)
for k, v in config.items():
  print("{} : {}".format(k, v))
  pathlib.Path('../figures/{}'.format(k)).mkdir(parents=True, exist_ok=True) 

m_li = [base_resnet50(model_path=config['resnet50'], num_classes=80, pretrained=True),\
   base_vit(config['vit'], 80, image_size=224, pretrained=True),\
      base_swin(config['swin'], 80, image_size=224, pretrained=True, requires_grad=True),\
         base_convnext(config['convnext'], 80, image_size=224, pretrained=True), \
           base_mlpmixer(config['mlpmixer'], num_classes=80, image_size=224, pretrained=True)]
# m_li2 = [BaseResnet(m_li[0], 80)]
p_li = ['/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_resnet50_base_best.pth.tar', \
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_vit_base_best.pth.tar', \
    '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_swin_base_best.pth.tar',\
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_convnext_base_best.pth.tar',\
      '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_mlpmixer_base_best.pth.tar' ,\
  ]

def get_model(index):
  path = p_li[index]
  model = m_li[index]
  di = torch.load(path)
  print(di['best_score'])
  print(di.keys())
  model.load_state_dict(di['state_dict'])
  return model


{'resnet50': 'resnetv2_50x3_bitm_in21k', 'vit': 'vit_base_patch16_224_in21k', 'vit-hybrid': 'vit_base_r50_s16_224_in21k', 'swin': 'swin_base_patch4_window7_224_in22k', 'convnext': 'convnext_base_in22k', 'mlpmixer': 'mixer_b16_224_in21k'}
resnet50 : resnetv2_50x3_bitm_in21k
vit : vit_base_patch16_224_in21k
vit-hybrid : vit_base_r50_s16_224_in21k
swin : swin_base_patch4_window7_224_in22k
convnext : convnext_base_in22k
mlpmixer : mixer_b16_224_in21k
Linear(in_features=768, out_features=80, bias=True)


/home/seongha/anaconda3/envs/MGSSL/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
i=1
key_list = list(config)
print(key_list[i])

vit


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model(i).to(device)
model = model.eval()
for i, (input, target) in tqdm(enumerate(test_loader)):
  img, path, inp = input
  target[target == 0] = 1
  target[target == -1] = 0
  feat_Var = torch.autograd.Variable(img).float().to(device)
  
  # output = model(feat_Var, None).detach()
  output = model(feat_Var, None).detach()
  # print(output.requires_grad, target.requires_grad)
  # print(output.shape, target.shape)
  AP.add(output, target)

# map = 100 * AP.value().mean()
# print(100 * AP.value())
# ap_li = 100 * AP.value()
print(AP.scores.shape)

{'mAP': tensor(29.3734), 'OF1': 0.35194173201084833, 'CF1': 0.15985432026494287}
dict_keys(['epoch', 'arch', 'state_dict', 'best_score'])


157it [2:03:36, 47.24s/it]

torch.Size([40137, 80])


In [7]:
import pathlib
for k, v in config.items():
  print("{} : {}".format(k, v))
  pathlib.Path('../figures/{}'.format(k)).mkdir(parents=True, exist_ok=True) 

resnet50 : resnetv2_50x3_bitm_in21k
vit : vit_base_patch16_224_in21k
vit-hybrid : vit_base_r50_s16_224_in21k
swin : swin_base_patch4_window7_224_in22k
convnext : convnext_base_in22k
mlpmixer : mixer_b16_224_in21k


In [8]:
classwise_img = dict({"class_index":[], "1":[], "2":[], "3":[], "-1":[], "-2":[], "-3":[]})
for i in range(AP.scores.shape[1]):
  score = AP.scores[:,i]
  target = AP.targets[:, i]
  score_true = score[torch.where(target == 1)]
  # print(torch.where(target==1))
  
  dataset_true = [test_dataset[each] for each in torch.where(target == 1)[0].tolist()]
  sorted, indices = torch.sort(score_true, dim=0, descending=True)
  
  print(score_true.shape)
  classwise_img["class_index"].append(i)
  classwise_img["1"].append(dataset_true[int(indices[0])][0][1])
  classwise_img["2"].append(dataset_true[int(indices[1])][0][1])
  classwise_img["3"].append(dataset_true[int(indices[2])][0][1])
  classwise_img["-1"].append(dataset_true[int(indices[-1])][0][1])
  classwise_img["-2"].append(dataset_true[int(indices[-2])][0][1])
  classwise_img["-3"].append(dataset_true[int(indices[-3])][0][1])
  
classwise_img
img_df = pd.DataFrame(data=classwise_img)
img_df.to_csv("../figures/{}/topandworst.csv".format(key_list[i]))

torch.Size([840])
torch.Size([491])
torch.Size([1832])
torch.Size([728])
torch.Size([799])
torch.Size([845])
torch.Size([341])
torch.Size([1292])
torch.Size([1961])
torch.Size([1114])
torch.Size([1121])
torch.Size([1048])
torch.Size([1828])
torch.Size([2912])


In [ ]:
plt.rcParams.update({'font.size': 8})
for i, row in img_df.iterrows():
  fig = plt.figure(figsize=(12, 12))
  for idx, (j, item) in enumerate(row.iteritems()):
    if j == "class_index":
      continue
    a = fig.add_subplot(2, 3, idx)
    img_id = '/home/seongha/LT-ML/data/coco/data/val2014/{}'.format(item)
    image = Image.open(img_id)  
    imgplot = plt.imshow(image)
    # a.axis("off")
    # a.set_title(names[i].split('(')[0], fontsize=30)
  
  plt.savefig('../figures/{}/topandworst_class_{}.png'.format(key_list[i], i), bbox_inches='tight', dpi=100)

/tmp/ipykernel_51055/868710965.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(12, 18))


# Classwise AP

In [ ]:
label_li = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
       'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
       'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']#voc
label_li = list(class_di.keys()) #coco
ap_li = 100 * AP.value()

In [ ]:

#classifier
ap_li = [75.4188, 76.3768, 66.3551, 79.1177, 45.1441, 62.4649, 83.6811, 85.9619,
        58.1541, 48.6313, 67.8912, 79.3836, 79.2103, 69.3254, 88.5846, 50.0462,
        26.8350, 57.0530, 75.6681, 71.1823]#FC
ap_li2 = [77.9105, 76.8534, 71.2095, 80.9140, 45.2674, 63.6200, 84.0961, 86.4019,
        59.8740, 52.9656, 68.0257, 80.5327, 79.3948, 71.0906, 88.7570, 49.5162,
        33.0532, 55.2915, 76.8210, 70.2609]#Transformer
ap_li_gcn = [44.9041, 51.7194, 41.8896, 52.5085, 31.0836, 42.8794, 75.6171, 61.9646,
        39.8540, 26.9209, 45.8847, 50.5602, 48.6032, 48.8849, 78.4147, 37.8425,
        15.7030, 46.4849, 45.6828, 53.3943]#gcn
ap_li_tc = [52.6595, 59.2412, 48.0880, 60.5390, 34.5672, 49.0508, 77.6743, 70.0045,
        42.0592, 32.3638, 49.0484, 57.0762, 55.9363, 55.3863, 81.9262, 41.5481,
        18.1342, 48.6349, 52.5640, 58.6664]#TC

#backbone;
ap_li_resnet50_fc_coco = [44.8806, 18.7181,  4.2505, 26.0276,  1.8891,  2.0793, 55.0725, 29.3584,
        21.6527, 13.7635, 34.6959, 50.7117,  4.5387,  9.8258, 10.3550, 42.5709,
        39.2878, 19.0187, 29.4253, 10.1510, 57.6950, 11.8762, 13.9065, 42.1499,
         4.6982, 33.9326, 10.8716, 15.9627, 32.3844, 19.4244, 61.4208, 29.5272,
         3.1157,  3.6808, 75.4917,  0.1942,  5.2710, 19.7860, 15.4992, 16.0211,
        15.0432,  3.9782, 21.1419,  8.8907, 30.8293,  1.9951, 32.1277, 21.3268,
        20.2985, 83.6436, 46.0174,  5.8537, 20.1276,  5.9066, 20.2521, 12.6364,
        54.7259, 21.3757,  3.1344,  6.6266,  3.5166,  3.2945,  3.5212, 31.7686,
        15.5602, 11.4469, 40.4510,  2.9287, 17.3079,  0.1819, 36.1461,  2.6347,
        30.3691, 31.7924, 28.6856, 21.0613, 17.1495, 20.7845,  3.5991, 73.5261]

In [ ]:
def sort_by_class_size(ap_li):
  Sorted_Ap = []
  for k, v in class_di.items():
        idx = label_li.index(k)
        print("{}, {}, {}".format(k, v, ap_li[idx]))
        Sorted_Ap.append( ap_li[idx])

  return Sorted_Ap

### sorted class wise ap

In [ ]:
ap_li = [(87.2317), (37.9691), (25.5833), (60.2952), (71.5306), (82.1195), (88.1945), (65.4069), (40.6149), (41.1147), (56.1550), (70.4887), (47.4118), (46.0852), (51.6309), (80.2644), (71.3544), (53.0769), (62.6369), (46.5251), (83.0326), (38.1768), (54.7562), (64.7367), (53.6059), (62.9250), (50.5712), (65.0720), (66.4795), (56.4693), (93.2486), (53.6870), (45.0067), (49.6613), (94.2364), (2.5078), (32.1601), (71.6280), (53.7966), (66.5539), (73.0900), (37.8341), (65.7573), (42.9252), (76.6879), (66.2664), (53.7973), (64.0184), (44.7371), (95.6273), (79.7539), (32.5730), (52.1021), (43.1948), (54.0491), (29.8070), (76.0650), (72.6686), (66.8284), (83.8258), (51.1537), (37.7894), (58.2430), (50.6816), (39.5533), (83.3577), (63.9864), (90.7062), (63.3839), (8.1543), (83.6346), (31.7384), (60.2249), (87.2840), (52.6824), (64.5797), (57.4210), (57.0662), (40.5110), (95.1418)]#resnet50

ap_li = [(45.3500), (18.2222), (13.6026), (23.4529), (42.3237), (49.2937), (22.8450), (20.3572), (15.8094), (14.2007), (14.6072), (27.6725), (19.2367), (23.4214), (28.1970), (49.6993), (35.2519), (13.4251), (39.2461), (25.9420), (25.2125), (14.7421), (30.5735), (27.2955), (15.7352), (13.5201), (27.8317), (40.5388), (10.5245), (13.2751), (37.1561), (7.5296), (24.1639), (19.6291), (54.0519), (0.7407), (17.6404), (13.3025), (11.5703), (24.3879), (34.7420), (19.7303), (27.4391), (16.8291), (38.1872), (25.5800), (30.7358), (24.3021), (3.6065), (84.7080), (46.3175), (18.6291), (19.6434), (14.5465), (21.8009), (3.2022), (25.4228), (42.3088), (15.8355), (59.7384), (23.9780), (16.0761), (36.3887), (28.1691), (7.4630), (52.5339), (13.9503), (54.7527), (24.7485), (1.0721), (46.4781), (5.2758), (30.5740), (40.9817), (26.0356), (31.9627), (14.5073), (20.7029), (12.1453), (82.5090)]#vit
ap_li = [(91.8544), (43.1256), (26.5690), (70.0659), (76.5766), (84.6905), (90.6705), (69.3488), (47.3465), (46.3260), (54.2423), (69.1731), (47.7069), (46.4438), (52.9935), (83.0113), (75.7541), (61.6797), (63.1336), (45.6008), (87.5212), (46.0696), (54.7016), (67.6697), (61.8478), (74.0051), (49.7442), (65.4748), (67.8092), (59.1135), (95.4002), (56.4176), (42.4934), (57.2041), (95.3415), (3.8831), (33.3568), (81.3636), (60.9955), (66.7404), (79.6752), (40.4530), (75.0224), (44.1766), (80.7608), (68.4051), (61.7054), (69.1169), (47.6977), (95.0259), (87.2624), (32.2685), (63.9545), (60.0900), (57.4585), (30.5509), (81.0403), (77.1382), (79.4230), (79.7268), (52.0464), (37.4560), (52.7913), (45.6313), (47.3443), (87.6463), (72.8188), (95.4763), (67.9169), (4.8832), (87.2957), (29.0972), (64.6534), (91.6069), (50.3383), (70.0396), (65.2082), (60.7164), (47.2639), (96.1118)]#swin



In [ ]:
idx = 2

In [ ]:
data_ = {"class_index": list(class_di.keys()),
"class_size": list(class_di.values()),
"AP": ap_li}

df = pd.DataFrame(data=data_)
df.to_csv("sorted_ap_{}.csv".format(key_list[i]))
df.reset_index()

,index,class_index,class_size,AP
0,0,49,45174,91.8544
1,1,22,8950,43.1256
2,2,18,8606,26.5690
3,3,27,8378,70.0659
4,4,26,6518,76.5766
...,...,...,...,...
75,75,55,673,70.0396
76,76,6,668,65.2082
77,77,48,481,60.7164
78,78,69,151,47.2639


### label distribution csv

In [ ]:
top = df.nlargest(8, 'AP')
print(top)
data_ = []
for i, j in zip(top.class_index, top.index):
  print(i)
  col = gt_labels[:, i]
  selected = gt_labels[np.isin(col, [1.0]), :]
  nonzero_cnt = (selected != 0).sum(1)
  cnter = Counter(nonzero_cnt)
  print(cnter)
  cnter = dict(sorted(cnter.items(),key = lambda i: i[0]))
  cnter['class_index'] = i
  cnter['rank'] = j
  data_.append(cnter)
df_top5 = pd.DataFrame(data=data_)
df_top5 = df_top5.fillna(0)
df_top5 = df_top5.set_index(['class_index', 'rank'])
df_top5.to_csv("label_distribution_top_{}.csv".format(name_map[idx]))

worst = df.nsmallest(8, 'AP')
data_ = []
for i, j in zip(worst.class_index, worst.index):
  print(i)
  col = gt_labels[:, i]
  selected = gt_labels[np.isin(col, [1.0]), :]
  nonzero_cnt = (selected != 0).sum(1)
  cnter = Counter(nonzero_cnt)
  print(cnter)
  cnter = dict(sorted(cnter.items(),key = lambda i: i[0]))
  cnter['class_index'] = i
  cnter['rank'] = j
  data_.append(cnter)
df_worst5 = pd.DataFrame(data=data_)
df_worst5 = df_worst5.fillna(0)
df_worst5 = df_worst5.set_index(['class_index', 'rank'])
df_worst5.to_csv("label_distribution_worst_{}.csv".format(name_map[idx]))

    class_index  class_size       AP
79           35         128  96.1118
67           19        1186  95.4763
30           61        2493  95.4002
34           67        2368  95.3415
49           34        1798  95.0259
0            49       45174  91.8544
73           38         821  91.6069
6            14        5028  90.6705
35
Counter({3: 42, 4: 27, 5: 22, 2: 17, 6: 10, 7: 3, 8: 3, 1: 3, 10: 1})
19
Counter({3: 290, 4: 239, 2: 210, 5: 160, 6: 88, 7: 67, 1: 36, 8: 31, 9: 28, 11: 14, 10: 12, 12: 9, 13: 1, 14: 1})
61
Counter({6: 395, 7: 375, 5: 352, 4: 346, 8: 267, 3: 219, 9: 179, 10: 110, 2: 85, 11: 81, 12: 45, 13: 20, 14: 8, 15: 5, 1: 4, 16: 2})
67
Counter({3: 1083, 2: 688, 4: 388, 5: 146, 6: 44, 7: 9, 1: 4, 9: 3, 8: 2, 11: 1})
34
Counter({1: 1234, 2: 407, 3: 116, 4: 27, 5: 8, 8: 3, 7: 2, 6: 1})
49
Counter({2: 15997, 3: 12123, 4: 7145, 5: 4298, 6: 2350, 7: 1388, 8: 747, 9: 436, 1: 238, 10: 228, 11: 124, 12: 57, 13: 27, 14: 9, 15: 6, 16: 1})
38
Counter({2: 209, 3: 181, 4: 126, 1: 8